<a href="https://colab.research.google.com/github/hyeonn-06/data_analysis/blob/main/rag_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-google-genai
!pip install langchain-community
!pip install chromadb
!pip install numpy

In [ ]:
import os
import getpass
import time

# --- 1. API 키 설정 ---
if "GOOGLE_API_KEY" not in os.environ:
    # Google Colab 등 환경에서는 직접 입력 대신 환경 변수 사용 권장
    try:
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")
    except Exception as e:
        print("API 키를 환경 변수로 설정하거나 직접 입력해주세요.")
        print(f"오류: {e}을")
        exit()

# API 키가 설정되었는지 최종 확인
if not os.getenv("GOOGLE_API_KEY"):
    print("오류: GOOGLE_API_KEY가 설정되지 않았습니다.")
    exit()
else:
    print("✅ Google API Key 설정 확인")

✅ Google API Key 설정 확인


In [ ]:
# --- 2. Langchain 관련 모듈 임포트 ---
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

print("✅ Langchain 모듈 임포트 완료")

✅ Langchain 모듈 임포트 완료


In [ ]:
# --- 3. 실제 청년 정책 데이터 준비 (예시) ---
# 출처: 웹 검색 정보 요약 (실제 신청 시 공식 공고 확인 필수!)
raw_documents = [
    {
        "id": "policy_doyak",
        "source": "온통청년 등 정부 웹사이트 (예시)",
        "text": """
        청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다.
        만 19세 이상 만 34세 이하 청년 중 개인소득(총급여 기준 7,500만원 이하) 및 가구소득(중위소득 180% 이하) 기준을 충족하면 가입할 수 있습니다.
        매월 70만원 한도 내에서 자유롭게 납입하면 정부가 월 최대 2만 4천원의 기여금을 지원하고, 이자소득에 비과세 혜택을 제공합니다.
        가입 기간은 5년 만기이며, 만기 시에는 시중은행 예적금 금리보다 높은 수준의 수익을 기대할 수 있습니다.
        가입 신청은 취급 은행 앱을 통해 가능하며, 가입 요건 충족 여부 확인 후 계좌 개설이 진행됩니다.
        """
    },
    {
        "id": "policy_naeil",
        "source": "고용노동부 내일채움공제 웹사이트 (예시)",
        "text": """
        청년내일채움공제는 중소·중견기업에 정규직으로 취업한 청년들의 장기근속과 목돈 마련을 지원하는 사업입니다.
        지원 대상은 만 15세 이상 34세 이하의 청년으로, 중소·중견기업에 정규직으로 취업하여 고용보험에 가입되어 있어야 합니다. (일부 학력, 소득 요건 존재 가능)
        청년이 2년간 매월 12만 5천원(총 300만원)을 적립하면, 정부와 기업이 함께 지원하여 만기 시 총 1,200만원(+이자)의 목돈을 마련할 수 있습니다. (기업 규모 등에 따라 유형 다름)
        신청은 워크넷-내일채움공제 누리집에서 온라인으로 신청하며, 청약 신청까지 완료해야 합니다.
        주의할 점은 생애 1회만 지원 가능하며, 정해진 기간 내에 신청해야 한다는 것입니다.
        """
    },
    {
        "id": "policy_housing_seoul",
        "source": "서울주거포털 (예시)",
        "text": """
        서울시 청년월세지원은 높은 주거비로 어려움을 겪는 서울 거주 청년들의 주거 안정을 돕기 위한 사업입니다.
        지원 대상은 서울시에 주민등록이 되어 있는 만 19세 ~ 39세 이하 청년 1인 가구로, 기준중위소득 150% 이하 요건을 충족해야 합니다.
        또한, 임차보증금 5천만원 이하 및 월세 60만원 이하의 건물에 월세로 거주하는 무주택자여야 합니다.
        선정된 청년에게는 월 20만원의 월세를 최대 12개월(240만원)까지 지원합니다.
        신청은 서울주거포털에서 온라인으로 접수하며, 소득 및 재산 기준 심사를 거쳐 대상자를 선정합니다. 매년 모집 시기와 세부 기준이 달라질 수 있으니 공고를 확인해야 합니다.
        """
    },
    {
        "id": "general_support_info",
        "source": "온통청년, 복지로 등 (예시)",
        "text": """
        정부와 지자체는 청년들의 자립과 성장을 돕기 위해 다양한 지원 정책을 펼치고 있습니다.
        주요 분야로는 일자리(취업 지원, 창업 지원), 주거(주택 구입/전세자금 대출 지원, 월세 지원), 교육(학자금 대출 지원, 역량 강화 프로그램), 복지·문화(청년수당, 문화생활 지원), 참여·권리(청년 정책 참여 기회 확대) 등이 있습니다.
        '온통청년' 웹사이트나 '복지로' 웹사이트에서 자신에게 맞는 정책을 검색하고 상세 정보를 확인할 수 있습니다.
        각 정책마다 지원 대상, 조건, 신청 방법 등이 다르므로 반드시 공식 공고를 꼼꼼히 확인하는 것이 중요합니다.
        """
    }
]

# Langchain Document 객체로 변환 (메타데이터에 출처 추가)
documents = [Document(page_content=doc['text'], metadata={'id': doc['id'], 'source': doc['source']}) for doc in raw_documents]
print(f"📚 총 {len(documents)}개의 실제 정책 정보 Document 준비 완료")

📚 총 4개의 실제 정책 정보 Document 준비 완료


In [ ]:
# --- 4. 문서 분할 (Text Splitting) ---
# 정책 내용이 길 수 있으므로 청크 크기를 적절히 조절
text_splitter = RecursiveCharacterTextSplitter(chunk_size=110, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
print(f"✂️ 문서를 {len(split_documents)}개의 청크로 분할 완료")
print(f"샘플 청크 (첫 번째): {split_documents[0].page_content}") # 분할된 내용 확인 (옵션)

✂️ 문서를 24개의 청크로 분할 완료
샘플 청크 (첫 번째): 청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다.


In [ ]:
# --- 5. 임베딩 모델 초기화 (Gemini) ---
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", task_type="retrieval_document")
print(f"🤖 임베딩 모델 ({embedding_model.model}) 초기화 완료")

🤖 임베딩 모델 (models/text-embedding-004) 초기화 완료


In [ ]:
# --- 6. 벡터 저장소 설정 (ChromaDB) 및 문서 저장 ---
print("💾 벡터 저장소(ChromaDB) 설정 및 문서 임베딩/저장 중...")
start_time = time.time()
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embedding_model,
    collection_name="youth_policy_rag_real" # 컬렉션 이름 변경
    # persist_directory="./chroma_db_policy" # 영구 저장을 원할 경우
)
end_time = time.time()
print(f"✅ 벡터 저장소에 문서 저장 완료 (소요 시간: {end_time - start_time:.2f}초)")

💾 벡터 저장소(ChromaDB) 설정 및 문서 임베딩/저장 중...
✅ 벡터 저장소에 문서 저장 완료 (소요 시간: 0.53초)


In [ ]:
# --- 7. 검색기(Retriever) 생성 ---
retriever = vectorstore.as_retriever(search_kwargs={'k': 2}) # 상위 1개 청크 검색
print(f"🔍 검색기(Retriever) 생성 완료 (k={retriever.search_kwargs['k']})")

🔍 검색기(Retriever) 생성 완료 (k=2)


In [ ]:
# --- 8. LLM 초기화 (Gemini) ---
# safety_settings에 사용할 Enum 타입 임포트
from google.generativeai.types import HarmCategory, HarmBlockThreshold

print("🔧 LLM 초기화 준비 중... safety settings 설정 적용")

try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0, # 약간의 유연성 부여
        convert_system_message_to_human=True,
        # safety_settings를 Enum 타입을 사용하여 정의
        safety_settings={
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
    )
    print(f"💡 LLM ({llm.model}) 초기화 완료 (Enum 기반 safety settings 적용)")
except ImportError:
    print("⚠️ 경고: 'google.generativeai.types'에서 Enum을 임포트할 수 없습니다.")
    print("   'pip install google-generativeai'가 제대로 설치되었는지 확인하세요.")
    print("   safety_settings 없이 LLM을 초기화합니다 (기본 설정 사용).")
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0,
        convert_system_message_to_human=True
    )
    print(f"💡 LLM ({llm.model}) 초기화 완료 (기본 safety settings 사용)")
except Exception as e:
    print(f"❌ LLM 초기화 중 예상치 못한 오류 발생: {e}")
    # 오류 발생 시 LLM 객체를 None으로 설정하거나, 기본값으로 재시도 고려
    llm = None # 또는 기본값 LLM으로 대체
    print("LLM 초기화 실패. 이후 단계에서 오류가 발생할 수 있습니다.")


# LLM 객체가 성공적으로 생성되었는지 확인 (선택적)
if llm is None:
    print("⛔ LLM 초기화에 실패하여 프로그램을 종료합니다.")
    exit()

🔧 LLM 초기화 준비 중... safety settings 설정 적용
💡 LLM (models/gemini-2.0-flash) 초기화 완료 (Enum 기반 safety settings 적용)


In [ ]:
# --- 9. 프롬프트 템플릿 정의 ---
prompt_template = """당신은 대한민국 청년 지원 정책 전문가입니다.
다음은 사용자의 질문과 관련된 정책 정보입니다. 이 정보를 바탕으로 사용자의 질문에 대해 친절하고 명확하게 한국어로 답변해주세요.
답변은 반드시 주어진 관련 문서 내용에 근거해야 합니다. 관련된 내용이 없다면, "제공된 정보만으로는 정확히 답변하기 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요." 라고 답변해주세요.

[사용자 질문]

{question}

[답변]
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
print("📝 프롬프트 템플릿 정의 완료")

📝 프롬프트 템플릿 정의 완료


In [ ]:
# --- 10. RAG 체인(Chain) 구성 (LCEL 사용) ---
def format_docs(docs):
    """검색된 Document 리스트를 문맥 문자열로 변환"""
    return "\n\n".join(f"출처: {doc.metadata.get('source', '알 수 없음')}\n내용: {doc.page_content}" for doc in docs)

# LCEL을 사용한 RAG 체인 구성
rag_chain = (
    RunnableParallel(
        # 질문을 기반으로 관련 문서를 검색하고 format_docs로 형식화
        context=retriever.pipe(format_docs),
        # 원본 질문 전달
        question=RunnablePassthrough()
    )
    | prompt
    | llm
    | StrOutputParser()
)
print("🔗 RAG 체인 구성 완료")

🔗 RAG 체인 구성 완료


In [ ]:
# --- 11. RAG 시스템 실행 및 결과 확인 함수 ---
def ask_policy_expert(query: str):
    """청년 정책 질문에 대해 RAG 체인을 실행하고 답변과 검색된 문서를 출력합니다."""
    print(f"\n❓ 사용자 질문: {query}")
    print("⏳ 정책 정보 검색 및 답변 생성 중...")

    start_time = time.time()
    # 검색 과정만 별도로 확인 (디버깅/이해 목적)
    try:
        retrieved_docs = retriever.invoke(query)
        print("\n📄 검색된 관련 문서 청크:")
        if retrieved_docs:
            for i, doc in enumerate(retrieved_docs):
                print(f"--- 문서 {i+1} (출처: {doc.metadata.get('source', '알 수 없음')}) ---")
                print(f"{doc.page_content[:200]}...") # 내용 일부 출력
                print("-" * (len(f"--- 문서 {i+1} ---") + len(doc.metadata.get('source', '알 수 없음'))+ 6))
        else:
            print("관련 문서를 찾지 못했습니다.")
    except Exception as e:
        print(f"문서 검색 중 오류 발생: {e}")
        retrieved_docs = [] # 오류 시 빈 리스트로 처리

    # 전체 RAG 체인 실행
    try:
        answer = rag_chain.invoke(query)
        end_time = time.time()
        print(f"\n💬 생성된 답변 (소요 시간: {end_time - start_time:.2f}초):")
        print(answer)
        return answer
    except Exception as e:
        end_time = time.time()
        print(f"\n❌ 답변 생성 중 오류 발생 (소요 시간: {end_time - start_time:.2f}초): {e}")
        # Gemini API 관련 상세 오류 피드백 확인
        if hasattr(e, 'response') and hasattr(e.response, 'prompt_feedback'):
             print(f"   - Prompt Feedback: {e.response.prompt_feedback}")
        if hasattr(e, 'response') and hasattr(e.response, 'candidates') and e.response.candidates:
             print(f"   - Finish Reason: {e.response.candidates[0].finish_reason}")
             print(f"   - Safety Ratings: {e.response.candidates[0].safety_ratings}")
        return "답변 생성 중 오류가 발생했습니다."


In [ ]:
# --- 12. 테스트 ---
if __name__ == "__main__":
    print("\n\n--- 실제 청년 정책 정보 기반 RAG 시스템 테스트 ---")

    ask_policy_expert("청년도약계좌 만들려면 소득 조건이 어떻게 돼?")
    ask_policy_expert("청년내일채움공제는 누가 신청할 수 있나요?")
    ask_policy_expert("서울에 사는 청년인데 월세 지원 받을 수 있을까?")
    ask_policy_expert("청년들을 위한 정책은 어떤 종류가 있나요?")
    ask_policy_expert("청년내일채움공제 만기되면 얼마 받아요?")
    ask_policy_expert("자동차 구매 지원 정책 알려줘") # 관련 없는 질문

    # ChromaDB 영구 저장을 사용했을 경우 정리 (필요 시 주석 해제)
    # try:
    #     print("\n🧹 컬렉션 삭제 시도...")
    #     vectorstore.delete_collection()
    #     print("✅ 컬렉션 삭제 완료.")
    # except Exception as e:
    #     print(f"컬렉션 삭제 중 오류 발생 (이미 삭제되었거나 권한 문제일 수 있음): {e}")



--- 실제 청년 정책 정보 기반 RAG 시스템 테스트 ---

❓ 사용자 질문: 청년도약계좌 만들려면 소득 조건이 어떻게 돼?
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------
--- 문서 2 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



💬 생성된 답변 (소요 시간: 1.05초):
청년도약계좌를 만들려면 소득 조건이 어떻게 되는지 궁금하시군요. 제공된 정보만으로는 정확히 답변하기 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요.

❓ 사용자 질문: 청년내일채움공제는 누가 신청할 수 있나요?
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------
--- 문서 2 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



💬 생성된 답변 (소요 시간: 0.98초):
제공된 정보만으로는 청년내일채움공제 신청 자격에 대해 정확히 답변하기 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요.

❓ 사용자 질문: 서울에 사는 청년인데 월세 지원 받을 수 있을까?
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------
--- 문서 2 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------


...


💬 생성된 답변 (소요 시간: 1.17초):
제공된 정보가 없어 답변이 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요.

❓ 사용자 질문: 청년내일채움공제 만기되면 얼마 받아요?
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------
--- 문서 2 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



💬 생성된 답변 (소요 시간: 1.98초):
제공된 정보만으로는 청년내일채움공제 만기 시 수령액을 정확히 답변하기 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요.

❓ 사용자 질문: 자동차 구매 지원 정책 알려줘
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 고용노동부 내일채움공제 웹사이트 (예시)) ---
청년내일채움공제는 중소·중견기업에 정규직으로 취업한 청년들의 장기근속과 목돈 마련을 지원하는 사업입니다....
----------------------------------------
--- 문서 2 (출처: 고용노동부 내일채움공제 웹사이트 (예시)) ---
청년내일채움공제는 중소·중견기업에 정규직으로 취업한 청년들의 장기근속과 목돈 마련을 지원하는 사업입니다....
----------------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



💬 생성된 답변 (소요 시간: 1.50초):
제공된 정보만으로는 자동차 구매 지원 정책에 대해 정확히 답변하기 어렵습니다. 온통청년 웹사이트 등 공식 정보를 확인해보세요.


In [ ]:
ask_policy_expert("서울시 청년월세지원의 지원대상에 대해서만 알려줘")


❓ 사용자 질문: 서울시 청년월세지원의 지원대상에 대해서만 알려줘
⏳ 정책 정보 검색 및 답변 생성 중...

📄 검색된 관련 문서 청크:
--- 문서 1 (출처: 고용노동부 내일채움공제 웹사이트 (예시)) ---
청년내일채움공제는 중소·중견기업에 정규직으로 취업한 청년들의 장기근속과 목돈 마련을 지원하는 사업입니다....
----------------------------------------
--- 문서 2 (출처: 온통청년 등 정부 웹사이트 (예시)) ---
청년도약계좌는 청년의 중장기 자산 형성을 지원하기 위한 정책형 금융상품입니다....
-------------------------------------
--- 문서 3 (출처: 온통청년, 복지로 등 (예시)) ---
정부와 지자체는 청년들의 자립과 성장을 돕기 위해 다양한 지원 정책을 펼치고 있습니다....
----------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



💬 생성된 답변 (소요 시간: 2.90초):
서울시 청년월세지원 사업의 지원 대상은 다음과 같습니다.

*   **소득 기준:** 신청일 기준 **본인** 가구의 기준 중위소득이 60% 이하이고, **최종 학력**을 기준으로 신청 연도에 따라 상이한 소득 기준을 충족해야 합니다. (구체적인 소득 기준은 매년 사업 공고 시 확인 필요)
*   **나이 기준:** 만 19세 ~ 39세 이하의 청년
*   **주거 기준:** 서울시에 거주하며, 임차보증금 5천만원 이하 및 월세 40만원 이하의 건물에 거주하는 무주택자 (단, 월세 40만원 초과 시에는 월세와 보증금 환산액을 합산한 금액이 70만원 이하인 경우도 가능)

**주의사항:**

*   자세한 소득 기준 및 기타 자격 요건은 매년 서울시에서 발표하는 청년월세지원 사업 공고를 통해 확인해야 합니다.
*   제공된 정보는 일반적인 내용이며, 실제 신청 시에는 반드시 최신 공고를 참고하여 정확한 자격 요건을 확인하시기 바랍니다.


'서울시 청년월세지원 사업의 지원 대상은 다음과 같습니다.\n\n*   **소득 기준:** 신청일 기준 **본인** 가구의 기준 중위소득이 60% 이하이고, **최종 학력**을 기준으로 신청 연도에 따라 상이한 소득 기준을 충족해야 합니다. (구체적인 소득 기준은 매년 사업 공고 시 확인 필요)\n*   **나이 기준:** 만 19세 ~ 39세 이하의 청년\n*   **주거 기준:** 서울시에 거주하며, 임차보증금 5천만원 이하 및 월세 40만원 이하의 건물에 거주하는 무주택자 (단, 월세 40만원 초과 시에는 월세와 보증금 환산액을 합산한 금액이 70만원 이하인 경우도 가능)\n\n**주의사항:**\n\n*   자세한 소득 기준 및 기타 자격 요건은 매년 서울시에서 발표하는 청년월세지원 사업 공고를 통해 확인해야 합니다.\n*   제공된 정보는 일반적인 내용이며, 실제 신청 시에는 반드시 최신 공고를 참고하여 정확한 자격 요건을 확인하시기 바랍니다.'